Analysis wheter or not the numerical solvers will converge toward the origin when $\mu<1$ 

In [2]:
using Plots
using LinearAlgebra
using DifferentialEquations
using Random

In [3]:
include("euler_method.jl")
include("DTM.jl")
include("lorentz_system.jl")
include("RK4.jl")
include("EulerCromer.jl")

EulerCromer (generic function with 1 method)

In [11]:
## parameter values
θ = 10.0; β = 8/3; μ = 1/2 # μ<1
tim = 1000
Δt = 0.001
r_vals = [100 250 500 750 1000]

1×5 Matrix{Int64}:
 100  250  500  750  1000

In [5]:
# Creates nr_of_dots initial conditions on a sphere with radius r

function Points_on_Sphere(nr_of_dots, r)
    dots = zeros(3, nr_of_dots)
    for i=1:nr_of_dots
        dot = rand(-1.0:0.001:1.0,(1,3))
        dot = r*dot./norm(dot)
        dots[:,i] = dot
    end
    return dots
end

Points_on_Sphere (generic function with 1 method)

In [6]:
# Percentage of converging dots

function Percentage_of_Convergence(r_vals, θ, μ, β, nr_of_dots, ẋ::Function, Δt::Float64)

    for r in r_vals

        dots = Points_on_Sphere(nr_of_dots, r)

        conv = zeros(nr_of_dots)
        tol = 0.1
        for i=1:nr_of_dots
            x̄=dots[:,i]
            while norm(x̄) > tol && norm(x̄) < 1000*r
                x̄=ẋ([θ,μ,β],x̄,Δt)
            end
            if norm(x̄) <= tol
                conv[i] = 1
            else 
                conv[i] = 0
            end
        end
        print(" & ", sum(conv)/nr_of_dots)
    end
end



Percentage_of_Convergence (generic function with 1 method)

In [7]:
r_vals = [100 250 500 750 1000]
Δt = 0.01
println(" \n Euler Forward \n ")
Percentage_of_Convergence(r_vals, θ, μ, β, 10000, EulerForward,Δt)
println(" \n Runge-Kutta 4 \n")
Percentage_of_Convergence(r_vals, θ, μ, β, 10000, RK4,Δt)
println("\n DTM \n")
Percentage_of_Convergence(r_vals, θ, μ, β, 10000, DTM,Δt)

 
 Euler Forward 
 


 & 0.0734

 & 0.0003

 & 0.0 & 0.0

 & 0.0 
 Runge-Kutta 4 



InterruptException: InterruptException:

In [ ]:
Δt = 0.001
r_vals = [250 500 1000 2500 5000 7500 10000]
println(" \n Euler Forward \n ")
Percentage_of_Convergence(r_vals, θ, μ, β, 10000, EulerForward,Δt)
println(" \n Runge-Kutta 4 \n")
Percentage_of_Convergence(r_vals, θ, μ, β, 10000, RK4,Δt)
println("\n DTM \n")
Percentage_of_Convergence(r_vals, θ, μ, β, 10000, DTM,Δt)

In [2]:
# Finds path for starting position x̄₀
function Path(ẋ::Function, Δt, tim, x̄₀, p)
    tsteps = Int(tim/Δt)
    x̄=x̄₀
    X = zeros(3,tsteps)
    for i=1:tsteps
        x̄= ẋ(p,x̄,Δt)
        X[:,i] = x̄
    end
    return X
end

Path (generic function with 1 method)

In [1]:
function Plot_Converging_Points(ẋ::Function, Δt, r, tim, p)
    nr_of_dots = 10
    dots = Points_on_Sphere(nr_of_dots, r)
    X_all_paths = zeros(nr_of_dots,3,Int(tim/Δt))
    for i=1:nr_of_dots
        x̄₀ = dots[:,i]
        X_all_paths[i,:,:] = Path(ẋ, Δt, tim, x̄₀, p)
    end

    plot(X_all_paths[1,1,:],X_all_paths[1,2,:],X_all_paths[1,3,:],legend=:bottomright)
    plot!(X_all_paths[2,1,:],X_all_paths[2,2,:],X_all_paths[2,3,:],legend=:bottomright)
    plot!(X_all_paths[3,1,:],X_all_paths[3,2,:],X_all_paths[3,3,:],legend=:bottomright)
    plot!(X_all_paths[4,1,:],X_all_paths[4,2,:],X_all_paths[4,3,:],legend=:bottomright)
    plot!(X_all_paths[5,1,:],X_all_paths[5,2,:],X_all_paths[5,3,:],legend=:bottomright)
    plot!(X_all_paths[6,1,:],X_all_paths[6,2,:],X_all_paths[6,3,:],legend=:bottomright)
    plot!(X_all_paths[7,1,:],X_all_paths[7,2,:],X_all_paths[7,3,:],legend=:bottomright)
    plot!(X_all_paths[8,1,:],X_all_paths[8,2,:],X_all_paths[8,3,:],legend=:bottomright)
    plot!(X_all_paths[9,1,:],X_all_paths[9,2,:],X_all_paths[9,3,:],legend=:bottomright)
    plot!(X_all_paths[10,1,:],X_all_paths[10,2,:],X_all_paths[10,3,:],legend=:bottomright)
    
end

Plot_Converging_Points (generic function with 1 method)

In [ ]:
Δt = 0.01
r = 100
tim = 1000
Plot_Converging_Points(RK4, Δt, r, tim, [θ,μ,β])

In [ ]:
Plot_Converging_Points(DTM, Δt, r, tim, [θ,μ,β])

In [3]:
# Percentage of converging dots for preexisting solvers

function Percentage_of_Convergence_Preex(r_vals, θ, μ, β, nr_of_dots, ẋ, Δt::Float64)

    for r in r_vals

        dots = Points_on_Sphere(nr_of_dots, r)

        conv = zeros(nr_of_dots)
        tol = 0.1
        for i=1:nr_of_dots
            x̄=dots[:,i]
            prob = ODEProblem(LorentzSystem,x̄,(0.0,r),[θ,μ,β])
            sol = solve(prob, ẋ, dt=Δt, adaptive=false)
            if norm(sol[end]) <= tol
                conv[i] = 1
            else 
                conv[i] = 0
            end
        end
        print(" & ", sum(conv)/nr_of_dots)
    end
end

Percentage_of_Convergence_Preex (generic function with 1 method)

In [5]:
# Preexisting Julia Solver
Δt = 0.01
#r_vals = [250 500 1000 2500 5000 7500 10000]
r_vals = [10000 25000]
nr_of_dots = 1000
Percentage_of_Convergence_Preex(r_vals, θ, μ, β, nr_of_dots, Vern9(), Δt)

UndefVarError: UndefVarError: Vern9 not defined

In [8]:
# RadauIIA5
# Rodas5 Rodas4P
Δt = 0.01
r_vals = [10000 25000]
nr_of_dots = 1000
Percentage_of_Convergence_Preex(r_vals, θ, μ, β, nr_of_dots, Rodas5(), Δt)

UndefVarError: UndefVarError: Rodas5 not defined

In [18]:
## parameter values
θ = 10.0; β = 8/3; μ = 23.5
time = 100
Δt = 0.001
nr_of_initial_conditions=5

# Fixed points
x̄_fix1 = [sqrt(8*22.5/3);
          sqrt(8*22.5/3);
          22.5]

x̄_fix2 = [-sqrt(8*22.5/3);
          -sqrt(8*22.5/3);
          22.5]



3-element Vector{Float64}:
 -7.745966692414834
 -7.745966692414834
 22.5

In [20]:
initial_vals = Points_on_Sphere(nr_of_initial_conditions, 2)
initial_vals1 = initial_vals .+ x̄_fix1
path = Path(RK4, Δt, time, initial_vals1[:,1], [θ,μ,β])

3×100000 Matrix{Float64}:
  6.37084   6.39348   6.4158    6.43782  …   7.73931   7.73931   7.73931
  8.651     8.6418    8.63264   8.62355      7.73943   7.73949   7.73955
 23.5918   23.5841   23.5766   23.5692      22.4922   22.4922   22.4921